In [1]:
from numpy import loadtxt
import pandas as pd
import seaborn as sns

In [2]:
events = pd.read_csv('../dataset/events_up_to_01062018.csv', low_memory = False)

In [3]:
#CREO UN DATA FRAME CON LAS PERSONAS PARA IR JOINEANDO MIS FEATURES AHI
persons = events['person'].to_frame().drop_duplicates()
persons.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38829 entries, 0 to 2341657
Data columns (total 1 columns):
person    38829 non-null object
dtypes: object(1)
memory usage: 606.7+ KB


In [4]:
#AGREGO UNA COLUMNA DE CANTIDAD
events['cantidad'] = 1

In [5]:
events.event.value_counts()

viewed product       1248124
brand listing         216312
visited site          204069
ad campaign hit       191388
generic listing       160176
searched products     130616
search engine hit     106406
checkout               65315
staticpage             11201
conversion              7091
lead                     983
Name: event, dtype: int64

In [6]:
events.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,cantidad
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [7]:
grouped_device = events.groupby(['person','device_type']).agg({'cantidad':'sum'})
grouped_device.sort_values(by='cantidad', ascending=False, inplace=True)
grouped_device = grouped_device.reset_index().drop_duplicates(keep='first', subset='person')
grouped_device.drop(columns='cantidad', inplace=True)
grouped_device.columns = ['person', 'favorite_device']
persons = pd.merge(persons, grouped_device, how='left', on='person')

In [8]:
persons['favorite_device'].value_counts()

Smartphone    20101
Computer      17462
Tablet          620
Unknown          59
Name: favorite_device, dtype: int64

In [9]:
checkouts = events.loc[events['event'] == 'checkout']
grouped_checkouts = checkouts.groupby(['person']).agg({'cantidad':'sum'})
grouped_checkouts.columns = ['cantidad_checkouts']
grouped_checkouts.reset_index(inplace=True)

In [10]:
persons = pd.merge(persons, grouped_checkouts, how='left', on='person')

In [11]:
conversions = events.loc[events['event'] == 'conversion']
grouped_conversions = conversions.groupby(['person']).agg({'cantidad':'count'})
grouped_conversions.columns = ['cantidad_conversions']

In [12]:
persons = pd.merge(persons, grouped_conversions, how='left', on='person')

In [13]:
persons = pd.get_dummies(persons, columns=['favorite_device'])
persons.set_index('person')
persons.to_csv('../features/features_gonza2.csv')